In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\didia\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "c:\Users\didia\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\didia\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\didia\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "c:\Users\didia\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

In [3]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

Pré-processamento da primeira meta 

In [4]:
df = pd.read_csv('factnews_dataset.csv')

#criar grupo treino e teste
train_df, test_df = train_test_split(df, test_size=0.7, random_state=42)

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train_df['tokens'] = train_df['sentences'].apply(lambda x: nltk.word_tokenize(str(x).lower()))

# Tokenizar sem stopwords
train_df['tokens'] = train_df['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])

divisão dos dados

In [5]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

'\n#criar grupo treino, validação e teste\ntrain_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[\'classe\'])\ntrain_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df[\'classe\'])\n\n#guardar conjuntos em memória\ntrain_df.to_csv("train.csv", index=False) \nval_df.to_csv("val.csv", index=False)\ntest_df.to_csv("test.csv", index=False)\n'

balanceamento dos dados

In [6]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

train_oversampling= pd.read_csv('train_oversampling.csv')
val_oversampling= pd.read_csv('val_oversampling.csv')
test_oversampling= pd.read_csv('test_oversampling.csv')

y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

In [7]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())
print("\nconjunto de treino + exemplos: ", y_train_oversampling.value_counts().sort_index())
print("\nconjunto de val + exemplos: ", y_val_oversampling.value_counts().sort_index())
print("\nconjunto de teste + exemplos: ", y_test_oversampling.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64

conjunto de treino + exemplos:  classe
-1    2375
 0    2375
 1    2375
Name: count, dtype: int64

conjunto de val + exemplos:  classe
-1    594
 0    594
 1    594
Name: count, dtype: int64

conjunto de teste + exemplos:  classe
-1    1273
 0    1273
 1    1273
Name: count, dtype: int64


In [8]:
from sklearn.utils import resample

def balancear_classes(conjunto):
  # Separar cada classe
  class_min = conjunto[conjunto['classe'] == -1]
  class_med = conjunto[conjunto['classe'] == 1]
  class_max = conjunto[conjunto['classe'] == 0]

  # Número da classe maior
  n_samples = len(class_max)

  # Replicar as classes menores até o tamanho da maior
  class_min_upsampled = resample(class_min, 
                              replace=True, 
                              n_samples=n_samples, 
                              random_state=42)

  class_med_upsampled = resample(class_med, 
                              replace=True, 
                              n_samples=n_samples, 
                            random_state=42)

  conjunto_oversampled = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

  # baralhar o conjunto 
  conjunto_oversampled = conjunto_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

  print(conjunto_oversampled['classe'].value_counts())

  return conjunto_oversampled

#train_oversampling.to_csv('train_oversampling.csv', index=False)
#val_oversampling = balancear_classes(val)
#test_oversampling= balancear_classes(test)

#val_oversampling.to_csv('val_oversampling.csv', index=False)
#test_oversampling.to_csv('test_oversampling.csv', index=False)


tokeinização

In [9]:
stop_words = set(stopwords.words('portuguese'))
def tokeinizar(train, val, test):

    #criar coluna tokens
    train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    train_oversampling['tokens'] = train_oversampling['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

    #remover stopwords
    train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    train_oversampling['tokens'] = train_oversampling['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

    #matizar
    #nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    train_oversampling['stems'] = train_oversampling['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

    nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    print(train['stems'].head())

    return train, val, test

train, val, test = tokeinizar(train, val, test)
train_oversampling, val_oversampling, test_oversampling = tokeinizar(train_oversampling, val_oversampling, test_oversampling)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object
0    [qued, ocorr, aliment, pux, cord, sent, contr,...
1    [repercuss, nega, cas, fez, bolsonar, ,, candi...
2    [brasil, lav, alm, após, decepcion, empat, col...
3    [final, encontr, ,, sen, tucan, diss, esp, gov...
4    [fiz, compromiss, líd, bas, hoj, líd, opos, ,,...
Name: stems, dtype: object


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [10]:
y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

# Construção dos modelos de representação

## TF-IDF - fred

In [11]:
def TF_IDF(train, val, test):
    tf_vect = TfidfVectorizer(ngram_range =(1, 2), min_df =3, max_df =0.5, max_features =500)

        #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tf_vect.fit_transform(text_train)
    X_val   = tf_vect.transform(text_val)
    X_test  = tf_vect.transform(text_test)
    return X_train, X_val, X_test

In [12]:
X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = TF_IDF(train,val, test)
X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec = TF_IDF(train_oversampling, val_oversampling, test_oversampling)

## CountVectorizer -didi

In [13]:
import optuna
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_count_vec(trial, train, val):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3)  # garante ngram_max >= ngram_min

    # CountVectorizer com parâmetros sugeridos
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(ngram_min, ngram_max),
        strip_accents='unicode',
        max_features=max_features
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)

    y_train = train['classe'] 
    y_val   = val['classe']

    # Treinar Logistic Regression
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    # Métrica: F1 macro
    score = f1_score(y_val, preds, average='macro')
    return score 

def count_vec(train, val, test):

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_count_vec(trial, train,val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)
    
    best_params=study.best_params
    print("Melhores parâmetros:", study.best_params)
    print("Melhor F1:", study.best_value)
    
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        strip_accents='unicode',
        max_features=best_params['max_features']
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    # Fit no treino, transform nos outros conjuntos
    X_train = c_vect.fit_transform(text_train)
    X_val = c_vect.transform(text_val)
    X_test = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [14]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train, val, test)
X_train_oversampling_count_vec, X_val_oversampling_count_vec, X_test_oversampling_count_vec= count_vec(train_oversampling, val_oversampling, test_oversampling)

[I 2025-11-13 01:50:33,433] A new study created in memory with name: no-name-b18ffb05-35c7-4e35-af0f-faef80f57f35
[I 2025-11-13 01:50:36,397] Trial 2 finished with value: 0.5237603313834623 and parameters: {'max_features': 521, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 2 with value: 0.5237603313834623.
[I 2025-11-13 01:50:36,545] Trial 5 finished with value: 0.517647901007312 and parameters: {'max_features': 629, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 2 with value: 0.5237603313834623.
[I 2025-11-13 01:50:36,751] Trial 0 finished with value: 0.541307907512978 and parameters: {'max_features': 1767, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 0 with value: 0.541307907512978.
[I 2025-11-13 01:50:37,323] Trial 4 finished with value: 0.5578194611967199 and parameters: {'max_features': 2725, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 4 with value: 0.5578194611967199.
[I 2025-11-13 01:50:37,566] Trial 1 finished with value: 0.5656060755070657 and parameters: {'max_features'

Melhores parâmetros: {'max_features': 1579, 'ngram_min': 1, 'ngram_max': 3}
Melhor F1: 0.6697064751341765


[I 2025-11-13 01:50:47,277] A new study created in memory with name: no-name-d8329e17-a60a-4e5e-8ccd-0273ce8e6440
[I 2025-11-13 01:50:51,536] Trial 6 finished with value: 0.5601691366705105 and parameters: {'max_features': 915, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 6 with value: 0.5601691366705105.
[I 2025-11-13 01:50:51,736] Trial 5 finished with value: 0.5526658953479552 and parameters: {'max_features': 2041, 'ngram_min': 2, 'ngram_max': 3}. Best is trial 6 with value: 0.5601691366705105.
[I 2025-11-13 01:50:52,080] Trial 4 finished with value: 0.5434527227464546 and parameters: {'max_features': 2218, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 6 with value: 0.5601691366705105.
[I 2025-11-13 01:50:52,874] Trial 3 finished with value: 0.5095761863308149 and parameters: {'max_features': 4231, 'ngram_min': 2, 'ngram_max': 2}. Best is trial 6 with value: 0.5601691366705105.
[I 2025-11-13 01:50:53,333] Trial 1 finished with value: 0.5343507359623395 and parameters: {'max_featu

Melhores parâmetros: {'max_features': 4713, 'ngram_min': 1, 'ngram_max': 1}
Melhor F1: 0.6952896987736915


## Glove -fred

In [15]:
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_tfidf(trial):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3) 
    min_df = trial.suggest_int("min_df", 1, 10)
    max_df = trial.suggest_float("max_df", 0.3, 0.9)

    tfidf_vect = TfidfVectorizer(
        ngram_range=(ngram_min, ngram_max),
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        strip_accents='unicode',
        token_pattern=r"(?u)\b\w+\b|[^\w\s]"
    )

    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tfidf_vect.fit_transform(text_train)
    X_val = tfidf_vect.transform(text_val)

    y_train = train['classe']
    y_val = val['classe']

    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    score = f1_score(y_val, preds, average='macro')
    return score

def tfidf_optuna(train, val, test):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_tfidf, n_trials=30)

    best_params = study.best_params
    print("Melhores parâmetros:", best_params)

    tfidf_vect = TfidfVectorizer(
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        min_df=best_params['min_df'],
        max_df=best_params['max_df'],
        max_features=best_params['max_features'],
        strip_accents='unicode',
        token_pattern=r"(?u)\b\w+\b|[^\w\s]"
    )

    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tfidf_vect.fit_transform(text_train)
    X_val = tfidf_vect.transform(text_val)
    X_test = tfidf_vect.transform(text_test)

    return X_train, X_val, X_test


In [16]:
X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = tfidf_optuna(train,val, test)
X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec = tfidf_optuna(train_oversampling, val_oversampling, test_oversampling)

[I 2025-11-13 01:51:04,395] A new study created in memory with name: no-name-af6b8f7a-12fb-4d01-bef2-c9aae47b2817
[I 2025-11-13 01:51:04,545] Trial 0 finished with value: 0.545130935729811 and parameters: {'max_features': 4373, 'ngram_min': 1, 'ngram_max': 1, 'min_df': 4, 'max_df': 0.7646089010758795}. Best is trial 0 with value: 0.545130935729811.
[I 2025-11-13 01:51:04,978] Trial 1 finished with value: 0.5400271529310162 and parameters: {'max_features': 1218, 'ngram_min': 1, 'ngram_max': 3, 'min_df': 7, 'max_df': 0.4481121131545343}. Best is trial 0 with value: 0.545130935729811.
[I 2025-11-13 01:51:05,361] Trial 2 finished with value: 0.5361755059172814 and parameters: {'max_features': 1134, 'ngram_min': 1, 'ngram_max': 3, 'min_df': 8, 'max_df': 0.5670504407632962}. Best is trial 0 with value: 0.545130935729811.
[I 2025-11-13 01:51:05,899] Trial 3 finished with value: 0.5475782148528766 and parameters: {'max_features': 3138, 'ngram_min': 1, 'ngram_max': 3, 'min_df': 8, 'max_df': 0.8

Melhores parâmetros: {'max_features': 561, 'ngram_min': 1, 'ngram_max': 1, 'min_df': 10, 'max_df': 0.31737701013310565}


[I 2025-11-13 01:51:21,135] Trial 0 finished with value: 0.5666016756770843 and parameters: {'max_features': 3384, 'ngram_min': 1, 'ngram_max': 2, 'min_df': 7, 'max_df': 0.365130965123919}. Best is trial 0 with value: 0.5666016756770843.
[I 2025-11-13 01:51:21,556] Trial 1 finished with value: 0.5287096446566014 and parameters: {'max_features': 3021, 'ngram_min': 1, 'ngram_max': 2, 'min_df': 9, 'max_df': 0.3787835310609651}. Best is trial 0 with value: 0.5666016756770843.
[I 2025-11-13 01:51:22,210] Trial 2 finished with value: 0.5426471454750078 and parameters: {'max_features': 2924, 'ngram_min': 1, 'ngram_max': 3, 'min_df': 5, 'max_df': 0.3346213445383555}. Best is trial 0 with value: 0.5666016756770843.
[I 2025-11-13 01:51:23,024] Trial 3 finished with value: 0.5301951247756355 and parameters: {'max_features': 4343, 'ngram_min': 1, 'ngram_max': 3, 'min_df': 1, 'max_df': 0.4737387223967725}. Best is trial 0 with value: 0.5666016756770843.
[I 2025-11-13 01:51:23,586] Trial 4 finished 

Melhores parâmetros: {'max_features': 3384, 'ngram_min': 1, 'ngram_max': 2, 'min_df': 7, 'max_df': 0.365130965123919}


## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [17]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])

    return X_train, X_val, X_test

In [18]:
model = carregar_word2vec()
X_train_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, val, test, 'stems', model)
X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec = word2vec(train_oversampling, val_oversampling, test_oversampling, 'stems', model)

(929606, 300)


## WORD2VEC treinado com os nossos dados

In [19]:
import numpy as np
import optuna
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

#vetor médio da frase
def vetor_frase_w2v(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# matriz dos embeddings
def prep_dados_w2v(dados, coluna, model):
    return np.vstack([vetor_frase_w2v(tokens, model) for tokens in dados[coluna]])

def objective_w2v(trial, train, val):
    # Hiperparâmetros 
    vector_size = trial.suggest_int("vector_size", 50, 300)
    window = trial.suggest_int("window", 3, 10)
    min_count = trial.suggest_int("min_count", 1, 5)
    sg = trial.suggest_categorical("sg", [0, 1])  # 0 = CBOW, 1 = Skip-gram
    epochs = trial.suggest_int("epochs", 5, 20)
    C = trial.suggest_float("C", 1e-3, 10, log=True)
    use_scaler = trial.suggest_categorical("use_scaler", [True, False])

    # Treina Word2Vec com os parâmetros sugeridos
    sentences = train["tokens"]
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,
        workers=4,
        epochs=epochs
    )

    # Transformar dados nos embeddings médios
    X_train = prep_dados_w2v(train, "tokens", model)
    X_val = prep_dados_w2v(val, "tokens", model)
    y_train = train["classe"].values
    y_val = val["classe"].values

    if use_scaler:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Modelo de classificação (Logistic Regression)
    clf = LogisticRegression(C=C, max_iter=500)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)

    return f1_score(y_val, preds, average="macro")

def word2vec_vec(train, val, test):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_w2v(trial, train, val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)

    best_params = study.best_params
    print("Melhores parâmetros:", best_params)
    print("Melhor F1:", study.best_value)

    best_model = Word2Vec(
        sentences=train["tokens"],
        vector_size=best_params["vector_size"],
        window=best_params["window"],
        min_count=best_params["min_count"],
        sg=best_params["sg"],
        workers=4,
        epochs=best_params["epochs"]
    )

    # Criar embeddings médios para todos os conjuntos
    X_train = prep_dados_w2v(train, "tokens", best_model)
    X_val = prep_dados_w2v(val, "tokens", best_model)
    X_test = prep_dados_w2v(test, "tokens", best_model)

    if best_params["use_scaler"]:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

    return X_train, X_val, X_test


In [20]:
X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained = word2vec_vec(train, val, test)
X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained= word2vec_vec(train_oversampling, val_oversampling, test_oversampling)

[I 2025-11-13 01:51:36,961] A new study created in memory with name: no-name-0f5c8fe1-72b8-4008-8929-1f071be82d0f
[I 2025-11-13 01:51:42,151] Trial 0 finished with value: 0.271047227926078 and parameters: {'vector_size': 117, 'window': 5, 'min_count': 3, 'sg': 1, 'epochs': 6, 'C': 0.007853554557048038, 'use_scaler': False}. Best is trial 0 with value: 0.271047227926078.
[I 2025-11-13 01:51:43,446] Trial 1 finished with value: 0.271047227926078 and parameters: {'vector_size': 203, 'window': 10, 'min_count': 3, 'sg': 1, 'epochs': 6, 'C': 0.006528666191206588, 'use_scaler': False}. Best is trial 0 with value: 0.271047227926078.
[I 2025-11-13 01:51:46,921] Trial 3 finished with value: 0.4753718663084852 and parameters: {'vector_size': 158, 'window': 8, 'min_count': 3, 'sg': 1, 'epochs': 9, 'C': 0.1266292466348613, 'use_scaler': False}. Best is trial 3 with value: 0.4753718663084852.
[I 2025-11-13 01:51:47,882] Trial 6 finished with value: 0.27463423725654645 and parameters: {'vector_size':

Melhores parâmetros: {'vector_size': 201, 'window': 4, 'min_count': 1, 'sg': 0, 'epochs': 20, 'C': 8.32641537265998, 'use_scaler': True}
Melhor F1: 0.5408646964721732


[I 2025-11-13 01:52:28,445] A new study created in memory with name: no-name-c9c9034b-c711-4104-a43d-de4afb8ab373
[I 2025-11-13 01:52:47,653] Trial 5 finished with value: 0.5836035839995114 and parameters: {'vector_size': 184, 'window': 9, 'min_count': 2, 'sg': 0, 'epochs': 10, 'C': 0.004466974781221183, 'use_scaler': False}. Best is trial 5 with value: 0.5836035839995114.
[I 2025-11-13 01:52:47,668] Trial 6 finished with value: 0.5829833631119572 and parameters: {'vector_size': 98, 'window': 5, 'min_count': 1, 'sg': 1, 'epochs': 10, 'C': 0.1964791626801931, 'use_scaler': False}. Best is trial 5 with value: 0.5836035839995114.
[I 2025-11-13 01:52:49,185] Trial 1 finished with value: 0.636032161671964 and parameters: {'vector_size': 129, 'window': 7, 'min_count': 3, 'sg': 0, 'epochs': 7, 'C': 0.00889598517719848, 'use_scaler': True}. Best is trial 1 with value: 0.636032161671964.
[I 2025-11-13 01:53:01,933] Trial 3 finished with value: 0.6488409404595951 and parameters: {'vector_size': 

Melhores parâmetros: {'vector_size': 216, 'window': 9, 'min_count': 4, 'sg': 0, 'epochs': 17, 'C': 0.0012844517425825812, 'use_scaler': False}
Melhor F1: 0.6656187485863588


# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [21]:
def objective_naive_MN(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    alpha = trial.suggest_float("alpha", 1e-3, 2.0, log=True)
    
    # Modelo
    clf = MultinomialNB(alpha=alpha)
    clf.fit(X_train, y_train)
    
    y_val_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_val_pred, average='weighted')  
    
    return f1

def Naive_Bayes_MN(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_MN(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print(f"Melhor F1 (validação): {study.best_value:.4f}")
    
    # Treinar o modelo final com o melhor alpha
    best_model = MultinomialNB(**study.best_params)
    best_model.fit(X_train, y_train)
    
    # Previsão no conjunto de teste
    y_test_pred = best_model.predict(X_test)
    
    return y_test_pred

In [22]:
from sklearn.metrics import f1_score

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [23]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB = Naive_Bayes_MN(X_train_count_vec,X_val_count_vec,X_test_count_vec,y_train,y_val)
print(metrics.classification_report(y_test, count_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

print("\ncom oversampling")
count_vec_over_predicted_NB = Naive_Bayes_MN(X_train_oversampling_count_vec,X_val_oversampling_count_vec,X_test_oversampling_count_vec,y_train_oversampling,y_val_oversampling)
print(metrics.classification_report(y_test_oversampling, count_vec_over_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_over_predicted_NB))

[I 2025-11-13 01:53:58,547] A new study created in memory with name: no-name-c8f2bab8-abf9-4842-a9f3-be9b9ff180b9
[I 2025-11-13 01:53:58,632] Trial 1 finished with value: 0.7967883970630032 and parameters: {'alpha': 0.08061444366540682}. Best is trial 1 with value: 0.7967883970630032.
[I 2025-11-13 01:53:58,644] Trial 3 finished with value: 0.7988139244481561 and parameters: {'alpha': 0.6421611341402095}. Best is trial 3 with value: 0.7988139244481561.
[I 2025-11-13 01:53:58,654] Trial 0 finished with value: 0.7991709867422968 and parameters: {'alpha': 0.11350918398907865}. Best is trial 0 with value: 0.7991709867422968.
[I 2025-11-13 01:53:58,656] Trial 5 finished with value: 0.7937552298987115 and parameters: {'alpha': 0.02571867002574687}. Best is trial 0 with value: 0.7991709867422968.
[I 2025-11-13 01:53:58,659] Trial 6 finished with value: 0.8006345325187125 and parameters: {'alpha': 0.29550244094989003}. Best is trial 6 with value: 0.8006345325187125.
[I 2025-11-13 01:53:58,659]

----COUNT VECTORIZER ----


[I 2025-11-13 01:53:58,742] Trial 8 finished with value: 0.7965894621414867 and parameters: {'alpha': 0.06752041844677995}. Best is trial 6 with value: 0.8006345325187125.
[I 2025-11-13 01:53:58,747] Trial 9 finished with value: 0.7938292666788604 and parameters: {'alpha': 0.006861775119224518}. Best is trial 6 with value: 0.8006345325187125.
[I 2025-11-13 01:53:58,757] Trial 11 finished with value: 0.7902003244108369 and parameters: {'alpha': 0.0019490781505063421}. Best is trial 6 with value: 0.8006345325187125.
[I 2025-11-13 01:53:58,760] Trial 10 finished with value: 0.8007643559307454 and parameters: {'alpha': 1.1692692779334497}. Best is trial 10 with value: 0.8007643559307454.
[I 2025-11-13 01:53:58,781] Trial 12 finished with value: 0.7983682448724285 and parameters: {'alpha': 0.25575583355899384}. Best is trial 10 with value: 0.8007643559307454.
[I 2025-11-13 01:53:58,794] Trial 13 finished with value: 0.7961807371272219 and parameters: {'alpha': 0.03657817046608279}. Best is 

Melhores hiperparâmetros encontrados:
{'alpha': 1.1692692779334497}
Melhor F1 (validação): 0.8008
              precision    recall  f1-score   support

          -1       0.72      0.76      0.74       417
           0       0.84      0.84      0.84      1273
           1       0.26      0.23      0.24       168

    accuracy                           0.77      1858
   macro avg       0.61      0.61      0.61      1858
weighted avg       0.76      0.77      0.76      1858

[[ 316   90   11]
 [ 105 1068  100]
 [  16  114   38]]

com oversampling


[I 2025-11-13 01:53:59,183] Trial 12 finished with value: 0.624195139311101 and parameters: {'alpha': 0.04959636003444992}. Best is trial 0 with value: 0.6891268013087232.
[I 2025-11-13 01:53:59,200] Trial 10 finished with value: 0.5480403458805844 and parameters: {'alpha': 0.0019040775077448422}. Best is trial 0 with value: 0.6891268013087232.
[I 2025-11-13 01:53:59,207] Trial 15 finished with value: 0.5987253185573506 and parameters: {'alpha': 0.029140077688549537}. Best is trial 0 with value: 0.6891268013087232.
[I 2025-11-13 01:53:59,209] Trial 9 finished with value: 0.5476306819182895 and parameters: {'alpha': 0.0024752552302671287}. Best is trial 0 with value: 0.6891268013087232.
[I 2025-11-13 01:53:59,222] Trial 13 finished with value: 0.6083190371747241 and parameters: {'alpha': 0.03708160194249976}. Best is trial 0 with value: 0.6891268013087232.
[I 2025-11-13 01:53:59,233] Trial 8 finished with value: 0.556281223790249 and parameters: {'alpha': 0.003787939280054039}. Best is 

Melhores hiperparâmetros encontrados:
{'alpha': 1.9387849617749588}
Melhor F1 (validação): 0.6971
              precision    recall  f1-score   support

          -1       0.76      0.80      0.78      1273
           0       0.57      0.75      0.65      1273
           1       0.70      0.44      0.54      1273

    accuracy                           0.66      3819
   macro avg       0.68      0.66      0.65      3819
weighted avg       0.68      0.66      0.65      3819

[[1013  190   70]
 [ 142  961  170]
 [ 170  547  556]]


In [24]:
print ("----TF-IDF ----")
tfidf_vec_predicted_NB= Naive_Bayes_MN(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NB))

print("\n com oversampling")
tfidf_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec, y_train_oversampling, y_val_oversampling) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))

[I 2025-11-13 01:53:59,502] A new study created in memory with name: no-name-f693ea1a-bb9f-41b0-9d62-98769c1cd461
[I 2025-11-13 01:53:59,564] Trial 0 finished with value: 0.755673354280673 and parameters: {'alpha': 0.3807611873028814}. Best is trial 0 with value: 0.755673354280673.
[I 2025-11-13 01:53:59,576] Trial 1 finished with value: 0.7582638282882935 and parameters: {'alpha': 0.2545124063301098}. Best is trial 1 with value: 0.7582638282882935.
[I 2025-11-13 01:53:59,580] Trial 2 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.015475727854694844}. Best is trial 2 with value: 0.7593235332092246.
[I 2025-11-13 01:53:59,596] Trial 5 finished with value: 0.7619920139669151 and parameters: {'alpha': 0.19465465144783933}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,611] Trial 4 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.0501774379265103}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,613] Tr

----TF-IDF ----


[I 2025-11-13 01:53:59,651] Trial 9 finished with value: 0.7606600986828669 and parameters: {'alpha': 0.16757134725510736}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,662] Trial 8 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.0048808107705348}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,684] Trial 10 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.002908113732422598}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,715] Trial 12 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.034292599355040136}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,717] Trial 11 finished with value: 0.7593235332092246 and parameters: {'alpha': 0.05827912444958128}. Best is trial 5 with value: 0.7619920139669151.
[I 2025-11-13 01:53:59,717] Trial 13 finished with value: 0.754703054254095 and parameters: {'alpha': 0.5304761953714479}. Best is trial

Melhores hiperparâmetros encontrados:
{'alpha': 0.19465465144783933}
Melhor F1 (validação): 0.7620
              precision    recall  f1-score   support

          -1       0.80      0.52      0.63       417
           0       0.77      0.96      0.86      1273
           1       0.67      0.01      0.02       168

    accuracy                           0.78      1858
   macro avg       0.75      0.50      0.50      1858
weighted avg       0.77      0.78      0.73      1858

[[ 216  201    0]
 [  44 1228    1]
 [  10  156    2]]

 com oversampling


[I 2025-11-13 01:54:00,093] Trial 10 finished with value: 0.6441462860274292 and parameters: {'alpha': 0.0021406139175066284}. Best is trial 0 with value: 0.6722693622924746.
[I 2025-11-13 01:54:00,114] Trial 11 finished with value: 0.6587964809592517 and parameters: {'alpha': 0.07196211688211081}. Best is trial 0 with value: 0.6722693622924746.
[I 2025-11-13 01:54:00,114] Trial 12 finished with value: 0.6588298900522627 and parameters: {'alpha': 0.0130719674131724}. Best is trial 0 with value: 0.6722693622924746.
[I 2025-11-13 01:54:00,127] Trial 15 finished with value: 0.642168710754768 and parameters: {'alpha': 0.0011256609474568473}. Best is trial 0 with value: 0.6722693622924746.
[I 2025-11-13 01:54:00,140] Trial 14 finished with value: 0.656864470722834 and parameters: {'alpha': 0.04771117842303776}. Best is trial 0 with value: 0.6722693622924746.
[I 2025-11-13 01:54:00,141] Trial 13 finished with value: 0.668937808637953 and parameters: {'alpha': 1.156350230496132}. Best is tria

Melhores hiperparâmetros encontrados:
{'alpha': 0.42044986968084547}
Melhor F1 (validação): 0.6803
              precision    recall  f1-score   support

          -1       0.76      0.77      0.76      1273
           0       0.50      0.83      0.62      1273
           1       0.70      0.24      0.36      1273

    accuracy                           0.61      3819
   macro avg       0.65      0.61      0.58      3819
weighted avg       0.65      0.61      0.58      3819

[[ 978  251   44]
 [ 134 1051   88]
 [ 176  791  306]]


In [25]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))

print ("oversampling")
word2vec_over_sampling_predicted_NB= naive_bayes_gs(X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))


[I 2025-11-13 01:54:00,342] A new study created in memory with name: no-name-80c49c07-1442-4306-8761-be124b8ab988
[I 2025-11-13 01:54:00,457] Trial 1 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 7.7428707460386e-08}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,457] Trial 2 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.6274988999220486e-09}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,457] Trial 0 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.1053700796863899e-11}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,475] Trial 3 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 8.918620266709172e-07}. Best is trial 1 with value: 0.4016961525191454.


----WORD2VEC ----


[I 2025-11-13 01:54:00,475] Trial 4 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.5849465941797246e-12}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,475] Trial 6 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.18492445068062e-07}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,475] Trial 5 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 2.6698348637013243e-08}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,491] Trial 7 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 3.375418822320465e-07}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,584] Trial 8 finished with value: 0.4016961525191454 and parameters: {'var_smoothing': 1.2044607660800132e-10}. Best is trial 1 with value: 0.4016961525191454.
[I 2025-11-13 01:54:00,592] Trial 9 finished with value: 0.4016961525191454 and paramete

Melhores hiperparâmetros encontrados:
{'var_smoothing': 7.7428707460386e-08}
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
           1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]
oversampling


[I 2025-11-13 01:54:01,225] Trial 1 finished with value: 0.4560574501374098 and parameters: {'var_smoothing': 1.2943987400236919e-09}. Best is trial 1 with value: 0.4560574501374098.
[I 2025-11-13 01:54:01,247] Trial 5 finished with value: 0.4560574501374098 and parameters: {'var_smoothing': 1.0187533802821353e-12}. Best is trial 1 with value: 0.4560574501374098.
[I 2025-11-13 01:54:01,254] Trial 2 finished with value: 0.4560574501374098 and parameters: {'var_smoothing': 2.182726745055361e-05}. Best is trial 1 with value: 0.4560574501374098.
[I 2025-11-13 01:54:01,277] Trial 0 finished with value: 0.4560574501374098 and parameters: {'var_smoothing': 1.8608119357196608e-10}. Best is trial 1 with value: 0.4560574501374098.
[I 2025-11-13 01:54:01,293] Trial 6 finished with value: 0.4560574501374098 and parameters: {'var_smoothing': 2.6202759466833022e-06}. Best is trial 1 with value: 0.4560574501374098.
[I 2025-11-13 01:54:01,294] Trial 4 finished with value: 0.4560574501374098 and parame

Melhores hiperparâmetros encontrados:
{'var_smoothing': 1.2943987400236919e-09}
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
           1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 410 667]
 [ 30  33 105]]


In [26]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_NB= naive_bayes_gs(X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_NB))

print("oversampling")
word2vec_oversampling_trained_predicted_NB= naive_bayes_gs(X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_NB))

[I 2025-11-13 01:54:02,129] A new study created in memory with name: no-name-0e8e8f98-cff2-49e9-abec-6d97f25d5e6f
[I 2025-11-13 01:54:02,204] Trial 1 finished with value: 0.4332751972427922 and parameters: {'var_smoothing': 3.181479933891035e-07}. Best is trial 1 with value: 0.4332751972427922.
[I 2025-11-13 01:54:02,223] Trial 5 finished with value: 0.4332751972427922 and parameters: {'var_smoothing': 3.723837377411133e-12}. Best is trial 1 with value: 0.4332751972427922.
[I 2025-11-13 01:54:02,223] Trial 2 finished with value: 0.4332751972427922 and parameters: {'var_smoothing': 5.258673784328246e-08}. Best is trial 1 with value: 0.4332751972427922.
[I 2025-11-13 01:54:02,235] Trial 0 finished with value: 0.4332751972427922 and parameters: {'var_smoothing': 2.961341122628346e-10}. Best is trial 1 with value: 0.4332751972427922.
[I 2025-11-13 01:54:02,235] Trial 4 finished with value: 0.4332751972427922 and parameters: {'var_smoothing': 8.464156576860832e-06}. Best is trial 1 with val

Melhores hiperparâmetros encontrados:
{'var_smoothing': 3.181479933891035e-07}
              precision    recall  f1-score   support

          -1       0.45      0.66      0.54       417
           0       0.85      0.32      0.47      1273
           1       0.13      0.60      0.22       168

    accuracy                           0.42      1858
   macro avg       0.48      0.53      0.41      1858
weighted avg       0.70      0.42      0.46      1858

[[274  43 100]
 [293 411 569]
 [ 39  28 101]]
oversampling


[I 2025-11-13 01:54:02,760] Trial 2 finished with value: 0.5861967367284407 and parameters: {'var_smoothing': 9.121403568322176e-05}. Best is trial 1 with value: 0.5861967367284407.
[I 2025-11-13 01:54:02,768] Trial 6 finished with value: 0.5861967367284407 and parameters: {'var_smoothing': 8.442218744391615e-07}. Best is trial 1 with value: 0.5861967367284407.
[I 2025-11-13 01:54:02,771] Trial 3 finished with value: 0.5861967367284407 and parameters: {'var_smoothing': 1.5066753350573997e-06}. Best is trial 1 with value: 0.5861967367284407.
[I 2025-11-13 01:54:02,775] Trial 0 finished with value: 0.5861967367284407 and parameters: {'var_smoothing': 9.109247296548664e-05}. Best is trial 1 with value: 0.5861967367284407.
[I 2025-11-13 01:54:02,782] Trial 4 finished with value: 0.5861967367284407 and parameters: {'var_smoothing': 1.25366301080702e-12}. Best is trial 1 with value: 0.5861967367284407.
[I 2025-11-13 01:54:02,787] Trial 7 finished with value: 0.5861967367284407 and parameters

Melhores hiperparâmetros encontrados:
{'var_smoothing': 0.0007800408532648814}
              precision    recall  f1-score   support

          -1       0.45      0.66      0.54       417
           0       0.85      0.32      0.47      1273
           1       0.13      0.60      0.22       168

    accuracy                           0.42      1858
   macro avg       0.48      0.53      0.41      1858
weighted avg       0.70      0.42      0.46      1858

[[274  43 100]
 [293 411 569]
 [ 39  28 101]]


In [27]:
#Glove
print ("----GLOVE ----")
glove_predicted_NB= naive_bayes_gs(X_train_glove, X_val_glove, X_test_glove, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, glove_predicted_NB))
print(metrics.confusion_matrix(y_test, glove_predicted_NB))

----GLOVE ----


NameError: name 'X_train_glove' is not defined

## SVM

In [28]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [29]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(
y_test, count_vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

print("\ncom oversampling")
count_vec_over_predicted_SVM = svm(X_train_oversampling_count_vec, X_val_oversampling_count_vec, X_test_oversampling_count_vec, y_train_oversampling, y_val_oversampling, n_trials=30)
print(metrics.classification_report(y_test_oversampling, count_vec_over_predicted_SVM))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_over_predicted_SVM))

[I 2025-11-13 01:55:28,543] A new study created in memory with name: no-name-fb4af387-d0a3-45df-9429-9c6e9f0d1220


----COUNT VECTORIZER ----


[I 2025-11-13 01:55:30,785] Trial 1 finished with value: 0.65764422131039 and parameters: {'C': 0.4671758860281262, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 1 with value: 0.65764422131039.
[I 2025-11-13 01:55:30,810] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 3.3427900094228766, 'kernel': 'poly', 'gamma': 'auto', 'degree': 5}. Best is trial 1 with value: 0.65764422131039.
[I 2025-11-13 01:55:30,936] Trial 3 finished with value: 0.271047227926078 and parameters: {'C': 0.13641763630293263, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.65764422131039.
[I 2025-11-13 01:55:31,446] Trial 5 finished with value: 0.5946632594446603 and parameters: {'C': 6.927975402821871, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best is trial 1 with value: 0.65764422131039.
[I 2025-11-13 01:55:31,726] Trial 0 finished with value: 0.5278430679381149 and parameters: {'C': 0.485204926504187, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best is 

Melhores hiperparâmetros encontrados:
{'C': 0.4671758860281262, 'kernel': 'linear', 'gamma': 'scale'}


[I 2025-11-13 01:55:40,992] A new study created in memory with name: no-name-d2e28a6c-8b63-4494-a67d-538f5aa89962


              precision    recall  f1-score   support

          -1       0.75      0.67      0.71       417
           0       0.82      0.89      0.85      1273
           1       0.27      0.15      0.19       168

    accuracy                           0.78      1858
   macro avg       0.61      0.57      0.58      1858
weighted avg       0.75      0.78      0.76      1858

[[ 281  127    9]
 [  79 1136   58]
 [  16  127   25]]

com oversampling


[I 2025-11-13 01:55:50,085] Trial 7 finished with value: 0.6522857001263389 and parameters: {'C': 2.4330142197598836, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 7 with value: 0.6522857001263389.
[I 2025-11-13 01:55:51,529] Trial 6 finished with value: 0.5654073581351042 and parameters: {'C': 5.319987538389064, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 7 with value: 0.6522857001263389.
[I 2025-11-13 01:55:54,823] Trial 3 finished with value: 0.5673638925581178 and parameters: {'C': 1.6031991695603132, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 7 with value: 0.6522857001263389.
[I 2025-11-13 01:55:55,257] Trial 2 finished with value: 0.521647957662429 and parameters: {'C': 2.0723413854978, 'kernel': 'poly', 'gamma': 'scale', 'degree': 3}. Best is trial 7 with value: 0.6522857001263389.
[I 2025-11-13 01:55:59,329] Trial 4 finished with value: 0.45724401602749404 and parameters: {'C': 0.4824754391616673, 'kernel': 'poly', 'gamma': 'scale', 'degree': 3}. Best is tria

Melhores hiperparâmetros encontrados:
{'C': 0.03440564595122038, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.84      0.74      0.79      1273
           0       0.52      0.83      0.64      1273
           1       0.72      0.37      0.49      1273

    accuracy                           0.65      3819
   macro avg       0.69      0.65      0.64      3819
weighted avg       0.69      0.65      0.64      3819

[[ 941  264   68]
 [ 101 1057  115]
 [  78  724  471]]


In [30]:
print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM= svm(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_SVM))

print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM_oversampling= svm(X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_SVM_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_SVM_oversampling))

[I 2025-11-13 01:56:58,800] A new study created in memory with name: no-name-beade782-90c7-4956-b6f1-cc658d410cdf


---- TF-IDF ----


[I 2025-11-13 01:57:00,308] Trial 7 finished with value: 0.5294004241372662 and parameters: {'C': 0.3464369309644731, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 7 with value: 0.5294004241372662.
[I 2025-11-13 01:57:00,318] Trial 4 finished with value: 0.5382299327291854 and parameters: {'C': 0.4609419620479682, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 4 with value: 0.5382299327291854.
[I 2025-11-13 01:57:01,168] Trial 2 finished with value: 0.5409567848592238 and parameters: {'C': 0.6933168454682551, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.5409567848592238.
[I 2025-11-13 01:57:01,196] Trial 5 finished with value: 0.5031314993382492 and parameters: {'C': 0.10834002692134755, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.5409567848592238.
[I 2025-11-13 01:57:01,538] Trial 6 finished with value: 0.5954805924095572 and parameters: {'C': 3.7133891205676353, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 6 with val

Melhores hiperparâmetros encontrados:
{'C': 8.26845694694628, 'kernel': 'linear', 'gamma': 'auto'}


[I 2025-11-13 01:57:08,161] A new study created in memory with name: no-name-aca92602-44ba-40ba-801a-53a998405e55


              precision    recall  f1-score   support

          -1       0.69      0.63      0.66       417
           0       0.81      0.88      0.84      1273
           1       0.29      0.14      0.19       168

    accuracy                           0.76      1858
   macro avg       0.59      0.55      0.56      1858
weighted avg       0.73      0.76      0.74      1858

[[ 263  149    5]
 [  98 1123   52]
 [  22  123   23]]
---- TF-IDF ----


[I 2025-11-13 01:57:15,893] Trial 3 finished with value: 0.6352366538983385 and parameters: {'C': 8.59045015882756, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 3 with value: 0.6352366538983385.
[I 2025-11-13 01:57:18,811] Trial 2 finished with value: 0.6480906603060672 and parameters: {'C': 1.686126762740429, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.6480906603060672.
[I 2025-11-13 01:57:19,026] Trial 5 finished with value: 0.6395929445035851 and parameters: {'C': 1.7949434600913086, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.6480906603060672.
[I 2025-11-13 01:57:24,900] Trial 6 finished with value: 0.47781320557889617 and parameters: {'C': 6.3202577896582826, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}. Best is trial 2 with value: 0.6480906603060672.
[I 2025-11-13 01:57:25,756] Trial 1 finished with value: 0.5256072333453886 and parameters: {'C': 2.0247503945046943, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 2 wit

KeyboardInterrupt: 

In [31]:
print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

print ("C/ oversampling")
word2vec_predicted_SVM_oversampling= svm(X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_SVM_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_SVM_oversampling))

[I 2025-11-13 01:58:46,111] A new study created in memory with name: no-name-ec09b9b4-2a4e-468d-87f9-a659d39b7e2c


----WORD2VEC ----


[I 2025-11-13 01:58:49,013] Trial 6 finished with value: 0.271047227926078 and parameters: {'C': 0.07187061322314273, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 6 with value: 0.271047227926078.
[I 2025-11-13 01:58:49,102] Trial 1 finished with value: 0.271047227926078 and parameters: {'C': 0.37545669617878075, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 6 with value: 0.271047227926078.
[I 2025-11-13 01:58:50,609] Trial 5 finished with value: 0.46050810369165934 and parameters: {'C': 2.8951189673999336, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 5 with value: 0.46050810369165934.
[I 2025-11-13 01:58:51,330] Trial 0 finished with value: 0.271047227926078 and parameters: {'C': 0.42492607272941557, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 5 with value: 0.46050810369165934.
[I 2025-11-13 01:58:51,347] Trial 4 finished with value: 0.271047227926078 and parameters: {'C': 0.020508090358132967, 'kernel': 'poly', 'gamma': 'au

Melhores hiperparâmetros encontrados:
{'C': 9.871815641594475, 'kernel': 'linear', 'gamma': 'scale'}


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
[I 2025-11-13 01:59:09,781] 

              precision    recall  f1-score   support

          -1       0.59      0.37      0.46       417
           0       0.74      0.93      0.83      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.72      1858
   macro avg       0.44      0.43      0.43      1858
weighted avg       0.64      0.72      0.67      1858

[[ 155  262    0]
 [  86 1187    0]
 [  22  146    0]]
C/ oversampling


[I 2025-11-13 01:59:44,201] Trial 1 finished with value: 0.6035837021889409 and parameters: {'C': 0.4288125541741209, 'kernel': 'poly', 'gamma': 'scale', 'degree': 3}. Best is trial 1 with value: 0.6035837021889409.
[I 2025-11-13 01:59:53,631] Trial 7 finished with value: 0.5548040892811187 and parameters: {'C': 0.9367374235572089, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 1 with value: 0.6035837021889409.
[I 2025-11-13 01:59:55,874] Trial 6 finished with value: 0.16666666666666666 and parameters: {'C': 0.5813849136939024, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 1 with value: 0.6035837021889409.
[I 2025-11-13 01:59:56,483] Trial 3 finished with value: 0.5694246468847518 and parameters: {'C': 0.28598963655566506, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 1 with value: 0.6035837021889409.
[I 2025-11-13 01:59:58,507] Trial 5 finished with value: 0.548775617422446 and parameters: {'C': 5.200370753016041, 'kernel': 'rbf', 'gamma': 'auto'}. Best is

Melhores hiperparâmetros encontrados:
{'C': 0.6395215591998831, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}
              precision    recall  f1-score   support

          -1       0.62      0.56      0.59      1273
           0       0.47      0.73      0.57      1273
           1       0.55      0.31      0.40      1273

    accuracy                           0.53      3819
   macro avg       0.55      0.53      0.52      3819
weighted avg       0.55      0.53      0.52      3819

[[718 407 148]
 [178 925 170]
 [254 624 395]]


In [ ]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_svm= svm(X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_svm))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_svm))

print("oversampling")
word2vec_trained_predicted_svm_over= svm(X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_svm_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_svm_over))

[I 2025-11-13 02:02:23,149] A new study created in memory with name: no-name-caaaacff-4eee-46a0-8d5a-564ee4bc789c
[I 2025-11-13 02:02:27,384] Trial 5 finished with value: 0.49823426290894046 and parameters: {'C': 0.32997100246459843, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 5 with value: 0.49823426290894046.
[I 2025-11-13 02:02:29,452] Trial 3 finished with value: 0.42679952550415184 and parameters: {'C': 0.6216029700021876, 'kernel': 'poly', 'gamma': 'scale', 'degree': 4}. Best is trial 5 with value: 0.49823426290894046.
[I 2025-11-13 02:02:29,860] Trial 6 finished with value: 0.41787638113991904 and parameters: {'C': 0.44537311508120203, 'kernel': 'poly', 'gamma': 'scale', 'degree': 5}. Best is trial 5 with value: 0.49823426290894046.
[I 2025-11-13 02:02:31,812] Trial 4 finished with value: 0.43993691493691495 and parameters: {'C': 2.0396742409740773, 'kernel': 'poly', 'gamma': 'auto', 'degree': 4}. Best is trial 5 with value: 0.49823426290894046.
[I 2025-11-13 02:02:31,892] 

In [ ]:
print ("----GLOVE ----")
glove_predicted_SVM= svm(X_train_glove, X_val_glove, X_test_glove, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, glove_predicted_SVM))
print(metrics.confusion_matrix(y_test, glove_predicted_SVM))

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

print ("c/ oversampling")
count_vec_predicted_KNN_oversampling= KNN(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_KNN_oversampling))

----COUNT VECTORIZER ----
Best k: 81
              precision    recall  f1-score   support

          -1       0.74      0.60      0.66       417
           0       0.79      0.94      0.85      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.51      0.51      1858
weighted avg       0.70      0.78      0.73      1858

[[ 250  167    0]
 [  80 1193    0]
 [   9  159    0]]
c/ oversampling


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 1
              precision    recall  f1-score   support

          -1       0.52      0.76      0.62      1273
           0       0.46      0.63      0.54      1273
           1       0.55      0.11      0.18      1273

    accuracy                           0.50      3819
   macro avg       0.51      0.50      0.44      3819
weighted avg       0.51      0.50      0.44      3819

[[965 265  43]
 [401 806  66]
 [473 665 135]]


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_KNN= KNN(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_KNN))

print ("c/oversampling")
tfidf_vec_predicted_KNN_oversampling= KNN(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_KNN_oversampling))

----TF-IDF ----
Best k: 181
              precision    recall  f1-score   support

          -1       0.74      0.60      0.66       417
           0       0.79      0.94      0.86      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.51      0.51      1858
weighted avg       0.71      0.78      0.74      1858

[[ 251  166    0]
 [  78 1195    0]
 [   9  159    0]]
c/oversampling


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 151
              precision    recall  f1-score   support

          -1       0.75      0.76      0.76      1273
           0       0.53      0.70      0.60      1273
           1       0.60      0.40      0.48      1273

    accuracy                           0.62      3819
   macro avg       0.62      0.62      0.61      3819
weighted avg       0.62      0.62      0.61      3819

[[973 179 121]
 [162 889 222]
 [169 599 505]]


In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

print ("c/oversampling")
word2vec_predicted_KNN_oversampling= KNN(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_KNN_oversampling))

----WORD2VEC ----
Best k: 11
              precision    recall  f1-score   support

          -1       0.62      0.27      0.37       417
           0       0.73      0.95      0.82      1273
           1       0.20      0.01      0.02       168

    accuracy                           0.71      1858
   macro avg       0.51      0.41      0.41      1858
weighted avg       0.65      0.71      0.65      1858

[[ 112  304    1]
 [  56 1210    7]
 [  13  153    2]]
c/oversampling
Best k: 31
              precision    recall  f1-score   support

          -1       0.54      0.47      0.50      1273
           0       0.48      0.39      0.43      1273
           1       0.41      0.53      0.46      1273

    accuracy                           0.46      3819
   macro avg       0.47      0.46      0.46      3819
weighted avg       0.47      0.46      0.46      3819

[[597 202 474]
 [252 496 525]
 [258 335 680]]


In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_KNN= KNN(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_KNN))
print(metrics.confusion_matrix(y_test, glove_predicted_KNN))

## Decison Tree

In [ ]:
#Criar um classificador de árvore de decisão baseado em entropia
from sklearn.tree import DecisionTreeClassifier

def decision_tree(X_train, y_train, X_val, y_val, X_test):
    alphas = np.linspace(0.000, 0.01, 20)  # Testar 20 valores entre 0 e 0.01
    best_alpha = 0
    best_acc = 0

    for alpha in alphas:
        clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=alpha, random_state=42)
        clf.fit(X_train, y_train)
        y_pred_val = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred_val)

        if acc > best_acc:
            best_acc = acc
            best_alpha = alpha

    print(f"Melhor alpha: {best_alpha:.4f} ")

    # Treinar modelo final com o melhor alpha
    final_clf = DecisionTreeClassifier(criterion='entropy', ccp_alpha=best_alpha, random_state=42)
    final_clf.fit(X_train, y_train)
    y_pred_test = final_clf.predict(X_test)

    return y_pred_test

In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_DT))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

print ("c/oversampling")
count_vec_predicted_DT_oversampling= decision_tree(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_DT_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_DT_oversampling))


----COUNT VECTORIZER ----
Melhor alpha: 0.0021 
              precision    recall  f1-score   support

          -1       0.73      0.63      0.68       417
           0       0.80      0.91      0.85      1273
           1       0.21      0.04      0.07       168

    accuracy                           0.77      1858
   macro avg       0.58      0.53      0.53      1858
weighted avg       0.73      0.77      0.74      1858

[[ 264  149    4]
 [  87 1163   23]
 [  11  150    7]]
c/oversampling
Melhor alpha: 0.0032 
              precision    recall  f1-score   support

          -1       0.80      0.69      0.74      1273
           0       0.50      0.54      0.52      1273
           1       0.52      0.56      0.54      1273

    accuracy                           0.60      3819
   macro avg       0.61      0.60      0.60      3819
weighted avg       0.61      0.60      0.60      3819

[[882 223 168]
 [117 683 473]
 [107 458 708]]


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= decision_tree(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_DT))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

print ("c/oversampling")
tfidf_vec_predicted_DT_oversampling= decision_tree(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_DT_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_DT_oversampling))


----TF-IDF ----
Melhor alpha: 0.0032 
              precision    recall  f1-score   support

          -1       0.68      0.66      0.67       417
           0       0.80      0.91      0.85      1273
           1       0.40      0.01      0.02       168

    accuracy                           0.77      1858
   macro avg       0.63      0.53      0.51      1858
weighted avg       0.73      0.77      0.73      1858

[[ 274  142    1]
 [ 115 1156    2]
 [  12  154    2]]
c/oversampling
Melhor alpha: 0.0089 
              precision    recall  f1-score   support

          -1       0.83      0.67      0.74      1273
           0       0.61      0.26      0.36      1273
           1       0.45      0.80      0.58      1273

    accuracy                           0.58      3819
   macro avg       0.63      0.58      0.56      3819
weighted avg       0.63      0.58      0.56      3819

[[ 855   28  390]
 [ 103  328  842]
 [  69  183 1021]]


In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_DT))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

print ("c/oversampling")
word2vec_predicted_DT_over= decision_tree(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_DT_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_DT_over))



----WORD2VEC ----
Melhor alpha: 0.0047 


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


              precision    recall  f1-score   support

          -1       0.41      0.30      0.35       417
           0       0.72      0.88      0.79      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.67      1858
   macro avg       0.38      0.39      0.38      1858
weighted avg       0.59      0.67      0.62      1858

[[ 126  291    0]
 [ 153 1120    0]
 [  26  142    0]]
c/oversampling
Melhor alpha: 0.0037 
              precision    recall  f1-score   support

          -1       0.42      0.49      0.46      1273
           0       0.39      0.53      0.45      1273
           1       0.42      0.19      0.26      1273

    accuracy                           0.41      3819
   macro avg       0.41      0.41      0.39      3819
weighted avg       0.41      0.41      0.39      3819

[[630 492 151]
 [410 679 184]
 [453 579 241]]


In [ ]:
#Glove
print ("---- GLOVE ----")
glove_predicted_DT= decision_tree(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_DT))
print(metrics.confusion_matrix(y_test, glove_predicted_DT))

## Random Florest

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, make_scorer

def objective(X_train, y_train, X_val, y_val, trial):
    n_estimators = trial.suggest_categorical('n_estimators', [50, 100, 200, 300])
    max_depth = trial.suggest_categorical('max_depth', [5, 10, 15, 20])
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
        random_state=42
    )

    clf.fit(X_train, y_train)

    predictions_rf = clf.predict(X_val)

    f1 = f1_score(y_val, predictions_rf, pos_label=1,average='macro')
    
    return f1

def random_florest(X_train, y_train, X_val, y_val, X_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial),
                   n_trials=50, n_jobs=-1)

    print("Melhores hiperparâmetros Random Forest:")
    print(study.best_params)

    best_rf = RandomForestClassifier(**study.best_params, random_state=42)
    best_rf.fit(X_train, y_train)

    predictions_rf = best_rf.predict(X_test)
    
    return predictions_rf


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_RF= random_florest(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_RF))
print(metrics.confusion_matrix(y_test, count_vec_predicted_RF))

print ("c/oversampling")
count_vec_predicted_RF_over= random_florest(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_RF_over))

[I 2025-11-13 01:43:51,007] A new study created in memory with name: no-name-a459fa79-5b88-45fb-9c47-6506fa72bbf5


----COUNT VECTORIZER ----


[I 2025-11-13 01:43:52,214] Trial 1 finished with value: 0.27818696949559824 and parameters: {'n_estimators': 50, 'max_depth': 20, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 1 with value: 0.27818696949559824.
[I 2025-11-13 01:43:52,251] Trial 3 finished with value: 0.27818696949559824 and parameters: {'n_estimators': 50, 'max_depth': 15, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 1 with value: 0.27818696949559824.
[I 2025-11-13 01:43:52,788] Trial 5 finished with value: 0.271047227926078 and parameters: {'n_estimators': 100, 'max_depth': 5, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 1 with value: 0.27818696949559824.
[I 2025-11-13 01:43:53,166] Trial 4 finished with value: 0.38263800100197937 and parameters: {'n_estimators': 100, 'max_depth': 15, 'max_features': 'sqrt', 'criterion': 'entropy'}. Best is trial 4 with value: 0.38263800100197937.
[I 2025-11-13 01:43:53,705] Trial 9 finished with value: 0.271047227926078 and parameters: 

Melhores hiperparâmetros Random Forest:
{'n_estimators': 50, 'max_depth': 20, 'max_features': None, 'criterion': 'gini'}


[I 2025-11-13 01:45:18,169] A new study created in memory with name: no-name-7e79c942-d2c7-4eec-8a3e-9e0f74a6561e


              precision    recall  f1-score   support

          -1       0.77      0.64      0.70       417
           0       0.80      0.94      0.86      1273
           1       0.36      0.02      0.04       168

    accuracy                           0.79      1858
   macro avg       0.64      0.53      0.54      1858
weighted avg       0.75      0.79      0.75      1858

[[ 266  150    1]
 [  70 1197    6]
 [   8  156    4]]
c/oversampling


[I 2025-11-13 01:45:19,943] Trial 3 finished with value: 0.6992281720698618 and parameters: {'n_estimators': 200, 'max_depth': 5, 'max_features': 'sqrt', 'criterion': 'gini'}. Best is trial 3 with value: 0.6992281720698618.
[I 2025-11-13 01:45:20,427] Trial 7 finished with value: 0.6860094103376161 and parameters: {'n_estimators': 200, 'max_depth': 15, 'max_features': 'log2', 'criterion': 'gini'}. Best is trial 3 with value: 0.6992281720698618.
[I 2025-11-13 01:45:20,801] Trial 9 finished with value: 0.6664953691346399 and parameters: {'n_estimators': 50, 'max_depth': 5, 'max_features': 'sqrt', 'criterion': 'entropy'}. Best is trial 3 with value: 0.6992281720698618.
[I 2025-11-13 01:45:21,196] Trial 8 finished with value: 0.6578222288752466 and parameters: {'n_estimators': 100, 'max_depth': 20, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 3 with value: 0.6992281720698618.
[I 2025-11-13 01:45:24,191] Trial 10 finished with value: 0.654196321436459 and parameters: {'n_e

Melhores hiperparâmetros Random Forest:
{'n_estimators': 200, 'max_depth': 5, 'max_features': 'sqrt', 'criterion': 'gini'}
              precision    recall  f1-score   support

          -1       0.77      0.74      0.75      1273
           0       0.50      0.76      0.61      1273
           1       0.69      0.37      0.48      1273

    accuracy                           0.62      3819
   macro avg       0.66      0.62      0.61      3819
weighted avg       0.66      0.62      0.61      3819

[[942 296  35]
 [133 965 175]
 [149 653 471]]


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_RF= random_florest(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_RF))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_RF))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_RF_over= random_florest(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_RF_over))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_RF= random_florest(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_RF))
print(metrics.confusion_matrix(y_test, word2vec_predicted_RF))

print ("c/oversampling")
word2vec_predicted_RF_over= random_florest(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_RF_over))

In [ ]:
#Glove
print ("---- GLOVE ----")
glove_predicted_DT= random_florest(X_train_glove, y_train, X_val_glove, y_val, X_test_glove, y_test) 
print(metrics.classification_report(y_test, glove_predicted_DT))
print(metrics.confusion_matrix(y_test, glove_predicted_DT))

## Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (100, 50)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        learning_rate_init=learning_rate_init,
        max_iter=300,
        random_state=42
    )

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

def neural_network (X_train, y_train, X_val, y_val, X_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)
    print("Melhores parâmetros:", study.best_params)

    best_params = study.best_params
    best_model = MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=300,
        random_state=42
    )

    best_model.fit(X_train, y_train)
    previsoes = best_model.predict(X_test)
    
    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NN= neural_network(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_NN))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NN))

print ("c/oversampling")
count_vec_predicted_NN_over= neural_network(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_NN_over))

In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_NN= neural_network(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_NN))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NN))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_NN_over= neural_network(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_NN_over))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NN= neural_network(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_NN))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NN))

print ("c/oversampling")
word2vec_predicted_NN_over= neural_network(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_NN_over))


In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_NN= neural_network(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_NN))
print(metrics.confusion_matrix(y_test, glove_predicted_NN))

## Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    # Hiperparâmetros a otimizar
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])  # compatíveis com L1 e L2
    max_iter = trial.suggest_int('max_iter', 100, 1000)

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=max_iter,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    f1 = f1_score(y_val, preds, average='macro')
    return f1


def logistic_regression(X_train, y_train, X_val, y_val, X_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)

    best_params = study.best_params
    best_model = LogisticRegression(
        **best_params,
        random_state=42
    )

    best_model.fit(X_train, y_train)

    previsoes = best_model.predict(X_test)

    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_LR= logistic_regression(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_LR))
print(metrics.confusion_matrix(y_test, count_vec_predicted_LR))

print ("c/oversampling")
count_vec_predicted_LR_over= logistic_regression(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_LR_over))

[I 2025-11-13 01:38:47,774] A new study created in memory with name: no-name-f804a6c1-a9ec-4956-92ee-a72dd1f88037
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:38:47,815] Trial 0 finished with value: 0.5759647633129582 and parameters: {'C': 0.5702975431962373, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 434}. Best is trial 0 with value: 0.5759647633129582.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator i

----COUNT VECTORIZER ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:48,033] Trial 2 finished with value: 0.6556576016190188 and parameters: {'C': 100.62436324687397, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 135}. Best is trial 2 with value: 0.6556576016190188.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:48,752] Trial 3 finished with value: 0.6504041189506938 and parameters: {'C': 162.82595003814305, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 567}. Best is trial 2 with value: 0.6556576016190188.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:49,360] 

Melhores parâmetros encontrados:
{'C': 3.4457245926565587, 'penalty': 'l1', 'solver': 'saga', 'max_iter': 277}


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:43:34,711] A new study created in memory with name: no-name-8b78f7ef-34bf-4cd4-ae02-5e635fa2a503
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:43:34,881] Trial 0 finished with value: 0.677824372328581 and parameters: {'C': 1.6684911551814994, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 894}. Best is trial 0 with value: 0.677824372328581.


              precision    recall  f1-score   support

          -1       0.77      0.64      0.70       417
           0       0.81      0.91      0.86      1273
           1       0.34      0.18      0.24       168

    accuracy                           0.78      1858
   macro avg       0.64      0.58      0.60      1858
weighted avg       0.76      0.78      0.77      1858

[[ 268  139   10]
 [  67 1159   47]
 [  13  125   30]]
c/oversampling


[I 2025-11-13 01:43:36,823] Trial 1 finished with value: 0.7002086643251745 and parameters: {'C': 0.01636145823939285, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 969}. Best is trial 1 with value: 0.7002086643251745.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:43:37,006] Trial 2 finished with value: 0.6204439095503163 and parameters: {'C': 41.268624677841466, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 568}. Best is trial 1 with value: 0.7002086643251745.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error wi

Melhores parâmetros encontrados:
{'C': 0.1084583216913827, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 263}
              precision    recall  f1-score   support

          -1       0.84      0.77      0.80      1273
           0       0.54      0.83      0.65      1273
           1       0.72      0.40      0.51      1273

    accuracy                           0.67      3819
   macro avg       0.70      0.67      0.66      3819
weighted avg       0.70      0.67      0.66      3819

[[ 979  219   75]
 [  96 1058  119]
 [  89  681  503]]


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_LR= logistic_regression(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_LR))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_LR))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_LR_over= logistic_regression(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_LR_over))



In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_LR= logistic_regression(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_LR))
print(metrics.confusion_matrix(y_test, word2vec_predicted_LR))

print ("c/oversampling")
word2vec_predicted_LR_over= logistic_regression(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_LR_over))

In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_LR= logistic_regression(X_train_glove, y_train, X_val_glove, y_val, X_test_glove, y_test) 
print(metrics.classification_report(y_test, glove_predicted_LR))
print(metrics.confusion_matrix(y_test, glove_predicted_LR))